In [ ]:
from pyconsolida.budget_reader import *
from pyconsolida.budget_reader import _get_valid_costo_rows, _read_raw_budget_sheet
from pathlib import Path
import pandas as pd
import flammkuchen as fl
import numpy as np

## Compute difference

In [ ]:
dec_al.

In [ ]:
data_dict = fl.load("/Users/vigji/Desktop/icop/exported_luigi_sum-fasi-False/230802_170104_python_data.h5")
numerical = ["quantita", "imp. unit."]
budgets_tot, budgets_dec = data_dict["budgets"], data_dict["budgets_dec"]

budgets_tot = budgets_tot.set_index(["commessa", "codice", "fase"]).drop(["costo u.", "imp.comp."], axis=1)
budgets_dec = budgets_dec.set_index(["commessa", "codice", "fase"]).drop(["costo u.", "imp.comp."], axis=1)

dec_al, tot_al = budgets_dec.align(budgets_tot)
dec_al.loc[:, numerical] = dec_al.loc[:, numerical].fillna(0)
tot_al.loc[:, numerical] = tot_al.loc[:, numerical].fillna(0)


print("Before dropping weird negatives: ", (tot_al["quantita"] - dec_al["quantita"]).sum())

problematic = dec_al[(dec_al["quantita"] > tot_al["quantita"]) & (tot_al["quantita"] > 0)].index

In [ ]:
dec_al.loc[problematic, :]

In [ ]:
tot_al.loc[problematic, :]

In [ ]:


dec_al = dec_al.drop(problematic)
tot_al = tot_al.drop(problematic)

print("before dropping all negatives: ", (tot_al["quantita"] - dec_al["quantita"]).sum())
negative_correct = dec_al[(dec_al["quantita"] > tot_al["quantita"])].index

dec_al = dec_al.drop(negative_correct)
tot_al = tot_al.drop(negative_correct)

print("dropping all negatives: ", (tot_al["quantita"] - dec_al["quantita"]).sum())

In [ ]:
dec_al, tot_al = budgets_dec.align(budgets_tot)
dec_al.loc[:, numerical] = dec_al.loc[:, numerical].fillna(0)
tot_al.loc[:, numerical] = tot_al.loc[:, numerical].fillna(0)

In [ ]:
problematic = dec_al[(dec_al["quantita"] > tot_al["quantita"]) & (tot_al["quantita"] > 0)].index

In [ ]:
dec_al.loc[problematic, :]

In [ ]:
tot_al.loc[problematic, :]

In [ ]:
dec_al, tot_al = budgets_dec.align(budgets_tot)
all_diffs = dec_al["quantita"] - tot_al["quantita"]

not_matching_2021 = set(budgets_dec.index) - set(budgets_tot.index)
negative_diffs = set(all_diffs[all_diffs < 0].index)
budgets_dec.drop(list(negative_diffs - not_matching_2021))

dec_al, tot_al = budgets_dec.align(budgets_tot)
numerical = ["quantita"]
dec_al.loc[:, numerical] = dec_al.loc[:, numerical].fillna(0)
tot_al.loc[:, numerical] = tot_al.loc[:, numerical].fillna(0)

not_matching_2021 = set(budgets_dec.index) - set(budgets_tot.index)
negative_diffs = set(all_diffs[all_diffs < 0].index)

tot_al = tot_al.drop(list(negative_diffs))
dec_al = dec_al.drop(list(negative_diffs))

In [ ]:
(tot_al["quantita"] - dec_al["quantita"]).sum()

In [ ]:
(tot_al["quantita"] - dec_al["quantita"]).sum()

In [ ]:
all_diffs = dec_al - 

In [ ]:
s = budgets_dec.loc[list(negative_diffs - not_matching_2021)]

In [ ]:
print("December 2021")
budgets_dec.drop(list(negative_diffs - not_matching_2021))

In [ ]:
print("Sum 2022")
budgets_tot.loc[list(negative_diffs - not_matching_2021)].head()

In [ ]:
all_diffs = tot_al[numerical] - dec_al[numerical]

In [ ]:
all_diffs["quantita"].sum()

In [ ]:
 17930866.01